LeNet implementation using mxnet

In [1]:
#implementing LeNet using mxnet

import d2l
from mxnet import np,npx,init,gluon,autograd
from mxnet.gluon import nn
npx.set_np()

#using  the sequential  class to  form the layers
net=nn.Sequential()

#adding each layer  to the network using prebuilt functions 
net.add( 
        #a convolutional layer
        nn.Conv2D(channels=6,kernel_size=5,padding=2,activation="sigmoid"),
        
        #a  maxpool  layer to preserve  the  spatial features
        nn.AvgPool2D(pool_size=2,strides=2),
        
        #repeat the previous 2 steps  again with  more channels  and no padding
        #same  pooling layer
        nn.Conv2D(channels=16,kernel_size=5,activation="sigmoid"),
        nn.AvgPool2D(pool_size=2,strides=2),
        
        #add fully  connected dense layers
        nn.Dense(120, activation='sigmoid'),
        nn.Dense(84, activation='sigmoid'),
        nn.Dense(10)
        )

X = np.random.uniform(size=(1, 1, 28, 28))
net.initialize()

for layer in net:
    X = layer(X)
    print(layer.name, 'output shape:\t', X.shape)
    
print("Done")

batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size=batch_size)

def evaluate_accuracy_gpu(net,data_iter,ctx=None):
    if not ctx:
        ctx=list(net.collect_params().values())[0].list_ctx()[0]
    metric=d2l.Accumulator(2)
    for  X,y in data_iter:
        X,y=X.as_in_context(ctx),y.as_int_context(ctx)
        metric.add(d2l.accuracy(net(X),y),y.size)
    return metric[0]/metric[1]

def train_ch5(net, train_iter, test_iter, num_epochs, lr, ctx=d2l.try_gpu()):
    
    net.initialize(force_reinit=True, ctx=ctx, init=init.Xavier())
    loss = gluon.loss.SoftmaxCrossEntropyLoss()
    trainer = gluon.Trainer(net.collect_params(),'sgd', {'learning_rate': lr})
    animator = d2l.Animator(xlabel='epoch', xlim=[0, num_epochs],legend=['train loss', 'train acc', 'test acc'])
    timer = d2l.Timer()
    for epoch in range(num_epochs):
        metric = d2l.Accumulator(3)
        for i, (X, y) in enumerate(train_iter):
            timer.start()
            # Here is the only difference compared to train_epoch_ch3
            X, y = X.as_in_context(ctx), y.as_in_context(ctx)
            with autograd.record():
                y_hat = net(X)
                l = loss(y_hat, y)
            l.backward()
            trainer.step(X.shape[0])
            metric.add(l.sum(), d2l.accuracy(y_hat, y), X.shape[0])
            timer.stop()
            train_loss, train_acc = metric[0]/metric[2], metric[1]/metric[2]
            if (i+1) % 50 == 0:
                animator.add(epoch + i/len(train_iter),(train_loss, train_acc, None))
    test_acc = evaluate_accuracy_gpu(net, test_iter)
    animator.add(epoch+1, (None, None, test_acc))
    
    print('loss %.3f, train acc %.3f, test acc %.3f' % (
    train_loss, train_acc, test_acc))
    print('%.1f examples/sec on %s' % (metric[2]*num_epochs/timer.sum(), ctx))

lr, num_epochs = 0.9, 10
train_ch5(net, train_iter, test_iter, num_epochs, lr)

conv0 output shape:	 (1, 6, 28, 28)
pool0 output shape:	 (1, 6, 14, 14)
conv1 output shape:	 (1, 16, 10, 10)
pool1 output shape:	 (1, 16, 5, 5)
dense0 output shape:	 (1, 120)
dense1 output shape:	 (1, 84)
dense2 output shape:	 (1, 10)
Done


Implementation of CNN using pytorch

In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets , transforms

from torch.utils.data import DataLoader

transformss = transforms.Compose([transforms.Resize((28,28)),transforms.ToTensor()])
trainset = datasets.FashionMNIST('MNIST_data/', download = True, train = True, transform = transformss)
testset = datasets.FashionMNIST('MNIST_data/', download = True, train = False, transform = transformss)
trainloader = DataLoader(trainset, batch_size = 600, shuffle = True)
testloader = DataLoader(testset, batch_size = 64, shuffle = True)

In [20]:
class Net(nn.Module):
    
    def  __init__(self):
        super().__init__()
        
        self.conv=nn.Sequential(
            nn.Conv2d(1,6,kernel_size=5,padding=0,stride=1),
            nn.ReLU(),
            nn.MaxPool2d(2,stride=2), #dim=14*14
            
            nn.Conv2d(6,16,kernel_size=5,padding=0,stride=1),
            nn.ReLU(),
            nn.MaxPool2d(2,stride=2), #dim=6*6
            
            nn.Conv2d(16,32,kernel_size=3,padding=0,stride=1),
            nn.ReLU(),
            #dim=3*3
        )
        
        self.linear=nn.Sequential(
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64,32),
            nn.ReLU(),
            nn.Linear(32,10),
            nn.LogSoftmax(dim=-1)
        )
    
    def forward(self,image):
        
        output=self.conv(image)
        output=output.view(output.size(0),-1)
        output=self.linear(output)
        
        return output

In [33]:
cnn=Net()
iterations=10
optimizer=optim.Adam(cnn.parameters(),lr=0.01)
loss_fn=nn.CrossEntropyLoss()

In [22]:
def train(cnn,iterations,optimizer,loss_fn,trainloader):
    
    for i in range(iterations):
        
        lossperiter=0.0
        
        for j, data in enumerate(trainloader):
            
            img,label=data
            
            optimizer.zero_grad()
            output=cnn(img)
            loss=loss_fn(output,label)
            loss.backward()
            optimizer.step()
            lossperiter+=loss.item()
        
        print(lossperiter/600)

In [31]:
def test_network(testloader,convNet,optimizer,lossfunction):
    
    total_correct = 0
    avg_loss = 0.0
    for i,data in enumerate(testloader):
        
        images,labels=data
        #calculate the output for each image
        output = convNet(images)
        #find the loss
        pred = output.detach().max(1)[1]
        total_correct += pred.eq(labels.view_as(pred)).sum()

    avg_loss /= len(testset)
    print('Accuracy '+ str((float(total_correct)*100) / len(testset)))


In [34]:
train(cnn,iterations,optimizer,loss_fn,trainloader)

0.17688492556413016
0.0890192973613739
0.07533800661563873
0.06728329683343569
0.06086659456292788
0.05752184684077899
0.05446715414524078
0.052070976694424946
0.05156468364099662
0.0506229654699564


In [35]:
test_network(testloader,cnn,optimizer,loss_fn)

Accuracy 87.85
